In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [21]:
# 적용 룰: 3가지 조건
# 기존 트랜잭션을 롤백하고 새로운 트랜잭션 시작
conn.rollback()

qry = """
     WITH ranked_loans AS (
  SELECT 
    *,
    LEAD("openedAt") OVER (
      PARTITION BY "userId", "institutionName" 
      ORDER BY "openedAt"
    ) AS next_opened_at,
    LEAD("status") OVER (
      PARTITION BY "userId", "institutionName" 
      ORDER BY "openedAt"
    ) AS next_status,
    LEAD("loanAmount") OVER (
      PARTITION BY "userId", "institutionName" 
      ORDER BY "openedAt"
    ) AS next_loan_amount,
    DATE_PART('day', ("dueAt"::timestamp - "openedAt"::timestamp)) AS total_loan_period,
    DATE_PART('day', ("lastRepaymentAt"::timestamp - "openedAt"::timestamp)) AS actual_repayment_period,
    DATE_PART('day', (LEAD("openedAt") OVER (
      PARTITION BY "userId", "institutionName" 
      ORDER BY "openedAt"
    )::timestamp - "lastRepaymentAt"::timestamp)) AS days_until_next_loan
  FROM credit_history
  ),
  analyzed_loans AS (
    SELECT 
      *,
      CASE 
        WHEN (
          (actual_repayment_period <= total_loan_period * 0.5 OR actual_repayment_period <= 2)
          AND
          (next_status = 'lost' OR (days_until_next_loan <= 3 AND days_until_next_loan IS NOT NULL))
          AND
          (next_loan_amount >= "loanAmount" OR next_loan_amount IS NULL)
        ) THEN 1
        ELSE 0
      END AS cheater_flag
    FROM ranked_loans
  )
SELECT 
  ch.*,
  analyzed_loans.cheater_flag AS "cheater_count"
FROM credit_history ch
LEFT JOIN analyzed_loans 
  ON ch.id = analyzed_loans.id;
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=column_names)
df.to_csv('./의도적_상환_꾼.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt   id  \
0 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00   90   
1 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00  104   
2 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00   98   
3 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00  101   
4 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00  100   

          bureau  reportedAt                                 institutionName  \
0            crc  2023-10-31  BRANCH INTERNATIONAL FINANCIAL SERVICE LIMITED   
1            crc  2024-09-30           CARBON FINANCE AND INVESTMENT LIMITED   
2  first-central  2019-12-24                Carbon Microfinance Bank Limited   
3  first-central  2022-10-24                Carbon Microfinance Bank Limited   
4  first-central  2023-03-22                Carbon Microfinance Bank Limited   

  accountNumber           loanDescription loanAmount repayment

In [17]:
# 적용 룰: 연속성 추가

# 기존 트랜잭션을 롤백하고 새로운 트랜잭션 시작
conn.rollback()

qry = """
     WITH ranked_loans AS (
      SELECT 
        *,
        LEAD("openedAt") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        ) AS "next_opened_at",
        LEAD("status") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        ) AS "next_status",
        LEAD("loanAmount") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        ) AS "next_loan_amount",
        DATE_PART('day', ("dueAt"::timestamp - "openedAt"::timestamp)) AS "total_loan_period",
        DATE_PART('day', ("lastRepaymentAt"::timestamp - "openedAt"::timestamp)) AS "actual_repayment_period",
        DATE_PART('day', (LEAD("openedAt") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        )::timestamp - "lastRepaymentAt"::timestamp)) AS "days_until_next_loan"
      FROM "credit_history"
    ),
    analyzed_loans AS (
      SELECT 
        *,
        CASE 
          WHEN (
            ("actual_repayment_period" <= "total_loan_period" * 0.5 OR "actual_repayment_period" <= 3)
            AND
            ("next_status" = 'lost' OR ("days_until_next_loan" <= 3 AND "days_until_next_loan" IS NOT NULL))
            AND
            ("next_loan_amount" >= "loanAmount" OR "next_loan_amount" IS NULL)
          ) THEN 1
          ELSE 0
        END AS "cheater_flag"
      FROM ranked_loans
    ),
    counted_cheaters AS (
      SELECT 
        "userId",
        "institutionName",
        "openedAt",
        "cheater_flag",
        COUNT(*) OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
          ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) - 1 AS "raw_cheater_streak"
      FROM analyzed_loans
    ),
    cheater_streaks AS (
      SELECT 
        *,
        GREATEST(0, "raw_cheater_streak") AS "cheater_streak"
      FROM counted_cheaters
    )
    SELECT 
      ch.*,
      analyzed_loans."cheater_flag" AS "cheater_count",
      cheater_streaks."cheater_streak"
    FROM "credit_history" ch
    LEFT JOIN analyzed_loans 
      ON ch."id" = analyzed_loans."id"
    LEFT JOIN cheater_streaks
      ON ch."userId" = cheater_streaks."userId"
      AND ch."institutionName" = cheater_streaks."institutionName"
      AND ch."openedAt" = cheater_streaks."openedAt";
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=column_names)
df.to_csv('./의도적_상환_꾼.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

PermissionError: [Errno 13] Permission denied: './의도적_상환_꾼.csv'

In [41]:
# 적용 룰: 연속성 추가 (streak 뺴고)

# 기존 트랜잭션을 롤백하고 새로운 트랜잭션 시작
conn.rollback()

qry = """
     WITH ranked_loans AS (
      SELECT 
        *,
        LEAD("openedAt") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        ) AS "next_opened_at",
        LEAD("status") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        ) AS "next_status",
        LEAD("loanAmount") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        ) AS "next_loan_amount",
        DATE_PART('day', ("dueAt"::timestamp - "openedAt"::timestamp)) AS "total_loan_period",
        DATE_PART('day', ("lastRepaymentAt"::timestamp - "openedAt"::timestamp)) AS "actual_repayment_period",
        DATE_PART('day', (LEAD("openedAt") OVER (
          PARTITION BY "userId", "institutionName" 
          ORDER BY "openedAt"
        )::timestamp - "lastRepaymentAt"::timestamp)) AS "days_until_next_loan"
      FROM "credit_history" 
      ORDER BY "openedAt" ASC, "institutionName" ASC
    ),
    analyzed_loans AS (
      SELECT 
        *,
        CASE 
          WHEN "next_status" = 'lost' OR (
            ("actual_repayment_period" <= "total_loan_period" * 0.5 OR "actual_repayment_period" <= 3)
            AND
            ("days_until_next_loan" <= 3 AND "days_until_next_loan" IS NOT NULL)
            AND
            ("next_loan_amount" >= "loanAmount" OR "next_loan_amount" IS NULL)
          ) THEN 1
          ELSE 0
        END AS "cheater_flag"
      FROM ranked_loans
    ),
    
    
cheater_streaks AS (
  SELECT 
    "id",
    "userId",
    "institutionName",
    "openedAt",
    "cheater_flag",
    LAG("cheater_flag") OVER (
      PARTITION BY "userId", "institutionName"
      ORDER BY "openedAt"
    ) AS "previous_cheater_flag",
    CASE 
      WHEN "cheater_flag" = 1 AND LAG("cheater_flag") OVER (
        PARTITION BY "userId", "institutionName"
        ORDER BY "openedAt"
      ) = 1 THEN 1
      ELSE 0
    END AS "cheater_streak"
  FROM analyzed_loans
)

SELECT 
  ch2.*,
  analyzed_loans."cheater_flag" AS "cheater_count",
  cheater_streaks."cheater_streak"
FROM "credit_history" ch2
LEFT JOIN analyzed_loans 
  ON ch2."id" = analyzed_loans."id"
LEFT JOIN cheater_streaks
  ON ch2."id" = cheater_streaks."id";
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=column_names)
df.to_csv('./의도적_상환_꾼.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt   id  \
0 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00   90   
1 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00  104   
2 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00   98   
3 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00   99   
4 2024-11-18 04:37:35.926548+00:00 2024-11-18 04:37:35.926548+00:00  101   

          bureau  reportedAt                                 institutionName  \
0            crc  2023-10-31  BRANCH INTERNATIONAL FINANCIAL SERVICE LIMITED   
1            crc  2024-09-30           CARBON FINANCE AND INVESTMENT LIMITED   
2  first-central  2019-12-24                Carbon Microfinance Bank Limited   
3  first-central  2022-10-24                Carbon Microfinance Bank Limited   
4  first-central  2022-10-24                Carbon Microfinance Bank Limited   

  accountNumber           loanDescription loanAmount repayment